## 2 Ground Control Stations (simple)

This is a first version. The two ground control stations are working in series. 

In [1]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes,clean
from simulators import Simulator, Gazebo, GazeboConfig, QGC
from plan import Plan
from oracle import Oracle,GCS
from config import Color

kill_processes()

## Create Plans

In [2]:
offsets = [  # east, north, up, heading
    (5, 5, 0, 90),
    (10, 0, 0, 45),
    (-5, -10, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]
n_vehicles = len(offsets)
side_lens = (5, 7, 4, 1, 2)  # must agree with n_vehicles
local_paths = [
    Plan.create_square_path(side_len=side_len, alt=5) for side_len in side_lens
]

plans = [Plan.basic(wps=path, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [3]:
homes = np.array([offset[:3] for offset in offsets])
global_paths = [
    local2global(np.array(path), home)
    for path, home in zip(local_paths, homes)
]
colors = 2*[Color.BLUE]+3*[Color.GREEN]  # must agree with n_vehicles
waypoints = {
    f"waypoint_{i+1}": {"pos": global_paths[i], "color": colors[i]}
    for i in range(n_vehicles)
}
plot_3d_interactive(
    waypoints, title="Simulation Markers", expand=[0.2, 0.2, 0.6], ground=-0.05
)

## Choose Simulator

In [4]:
# simulator = Simulator(name="NONE", offsets=offsets, plans=plans)

# simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))


gazebo_config = GazeboConfig(world_path="simulators/gazebo/worlds/runway.world",
                            models=n_vehicles * ["iris"],
                            colors=colors,
                            markers = waypoints)

simulator = Gazebo(offsets=offsets,plans=plans,config=gazebo_config)

In [5]:
simulator

name='gazebo'
offsets=[(5, 5, 0, 90), (10, 0, 0, 45), (-5, -10, 0, 225), (-15, 0, 0, 0), (0, -20, 0, 0)]
info={'models': ['iris', 'iris', 'iris', 'iris', 'iris'], 'colors': [blue, blue, green, green, green], 'markers': {'waypoint_1': {'color': blue}, 'waypoint_2': {'color': blue}, 'waypoint_3': {'color': green}, 'waypoint_4': {'color': green}, 'waypoint_5': {'color': green}}, 'world_path': 'simulators/gazebo/worlds/runway_updated.world'}

Launch Simulator

In [5]:
sysids = list(range(1,n_vehicles+1))
simulator.launch()
orac = Oracle(sysids)
blue_gcs = GCS(sysids[:2],name='blue 🟦')
green_gcs = GCS(sysids[2:],name = 'green 🟩')

while len(orac.conns):
    blue_gcs.gather_broadcasts()
    green_gcs.gather_broadcasts()
    blue_gcs.save_pos()
    green_gcs.save_pos() 
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)
clean()

🚀 Vehicle 1 launched (PID 275130)
🚀 Vehicle 1 logic launched (PID 275131)
🚀 Vehicle 2 launched (PID 275132)
🚀 Vehicle 2 logic launched (PID 275133)
🚀 Vehicle 3 launched (PID 275134)
🚀 Vehicle 3 logic launched (PID 275135)
🚀 Vehicle 4 launched (PID 275136)
🚀 Vehicle 4 logic launched (PID 275137)
🚀 Vehicle 5 launched (PID 275138)
🚀 Vehicle 5 logic launched (PID 275139)
🔗 UAV logic 1 is connected to Oracle ⚪
🔗 UAV logic 2 is connected to Oracle ⚪
🔗 UAV logic 3 is connected to Oracle ⚪
🔗 UAV logic 4 is connected to Oracle ⚪
🔗 UAV logic 5 is connected to Oracle ⚪
🔗 UAV logic 1 is connected to GCS blue 🟦
🔗 UAV logic 2 is connected to GCS blue 🟦
🔗 UAV logic 3 is connected to GCS green 🟩
🔗 UAV logic 4 is connected to GCS green 🟩
🔗 UAV logic 5 is connected to GCS green 🟩
✅ Vehicle 4 terminated
✅ Vehicle 5 terminated
✅ Vehicle 3 terminated
✅ Vehicle 1 terminated
✅ Vehicle 2 terminated


## Execute Plan

In [6]:
blue_paths = {}
for sysid,path in blue_gcs.paths.items():
    clean_path =[x for x in path if x is not None]
    blue_paths[f"uav_{sysid}"] = {"pos": clean_path, "color": colors[sysid-1]}

In [7]:
plot_3d_interactive(
    blue_paths, title="blue_gcs", expand=[0.2, 0.2, 0.6], ground=-0.05
)

In [8]:
green_paths = {}
for sysid,path in green_gcs.paths.items():
    clean_path =[x for x in path if x is not None]
    green_paths[f"uav_{sysid}"] = {"pos": clean_path, "color": colors[sysid-1]}

plot_3d_interactive(
    green_paths, title="green_gcs", expand=[0.2, 0.2, 0.6], ground=-0.05
)